In [1]:
import os
import re
import pandas as pd
from Bio import SeqIO
from io import StringIO
from tqdm import tqdm

In [2]:
def parse_gbk(isolate):
    input_file = f"prokka/{isolate}/PROKKA_04282025.gbk"
    fixed_lines = []

    with open(input_file, "r") as infile:
        for line in infile:
            if line.startswith("LOCUS"):
                # Match Prokka-style: NODE_XXX_length_YYY
                match = re.match(r"LOCUS\s+(\S+)_length_(\d+)", line)
                if match:
                    locus_id = match.group(1)
                    length = match.group(2)
                else:
                    # fallback for general LOCUS lines
                    fallback = re.match(r"LOCUS\s+(\S+).*?(\d+)\s+bp", line)
                    if fallback:
                        locus_id = fallback.group(1)
                        length = fallback.group(2)
                    else:
                        raise ValueError(f"Can't parse LOCUS line: {line}")
                # Standard GenBank LOCUS line format
                fixed_line = f"LOCUS       {locus_id:<16}{length:>11} bp    DNA     linear       01-JAN-2025\n"
                fixed_lines.append(fixed_line)
            else:
                fixed_lines.append(line)

    fixed_gbk_stream = StringIO("".join(fixed_lines))
    records = list(SeqIO.parse(fixed_gbk_stream, "genbank"))

    return records

In [3]:
genes_by_contig = pd.DataFrame()
for iso in tqdm([d for d in os.listdir('prokka') if '.' not in d]):
    records = parse_gbk(iso)
    contigs = [rec.id for rec in records]
    genes = [[f.qualifiers['locus_tag'] for f in rec.features if f.type=='CDS'] for rec in records]
    temp = pd.DataFrame({'Isolate':iso, 'contig':contigs, 'genes':genes})
    genes_by_contig = pd.concat([genes_by_contig, temp], axis=0)
genes_by_contig = genes_by_contig.explode('genes')
genes_by_contig['locus_tag'] = genes_by_contig['genes'].apply(lambda x: x[0] if isinstance(x, list) and len(list(x))==1 else None)
genes_by_contig

100%|███████████████████████████████████████████████████████████████| 299/299 [01:01<00:00,  4.84it/s]


,Isolate,contig,genes,locus_tag
0,UNY170,contig_1_RagTag,[MNAEDFLO_00001],MNAEDFLO_00001
0,UNY170,contig_1_RagTag,[MNAEDFLO_00002],MNAEDFLO_00002
0,UNY170,contig_1_RagTag,[MNAEDFLO_00003],MNAEDFLO_00003
0,UNY170,contig_1_RagTag,[MNAEDFLO_00004],MNAEDFLO_00004
0,UNY170,contig_1_RagTag,[MNAEDFLO_00005],MNAEDFLO_00005
...,...,...,...,...
75,UWI276,NODE_6,[FGBJGDLG_01305],FGBJGDLG_01305
75,UWI276,NODE_6,[FGBJGDLG_01306],FGBJGDLG_01306
75,UWI276,NODE_6,[FGBJGDLG_01307],FGBJGDLG_01307
75,UWI276,NODE_6,[FGBJGDLG_01308],FGBJGDLG_01308


In [4]:
isolates = [d.strip('.gff') for d in os.listdir('pangenomes/roary/gff') if '.gff' in d]
#gene_presence_absence = pd.read_csv('pangenomes/roary/roary/gene_presence_absence.csv')[['Gene']+isolates]
gene_presence_absence = pd.read_csv('pangenomes/panaroo_no_merge/gene_presence_absence_roary.csv')[['Gene']+isolates]
gene_presence_absence_melt = gene_presence_absence.melt(id_vars='Gene', 
                                                        value_vars=[col for col in gene_presence_absence.columns if col!='Gene'],
                                                        var_name='Isolate',
                                                        value_name='locus_tag'
                                                   ).dropna(
                                                   ).reset_index(drop=True)
gene_presence_absence_melt

/tmp/ipykernel_721403/1065622485.py:3: DtypeWarning: Columns (17,18,19,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,38,39,40,41,42,44,45,46,48,49,50,51,52,53,55,56,57,58,59,61,63,64,65,66,67,68,69,72,76,78,79,81,85,89,90,91,92,93,94,95,96,97,99,100,103,104,105,106,108,109,110,113,114,115,116,117,118,119,120,121,123,124,125,126,127,129,131,132,134,137,139,140,141,142,143,144,145,146,147,149,150,151,153,154,156,157,158,160,161,164,166,167,170,173,175,177,178,179,180,181,182,183,184,186,187,190,191,193,198,200,202,206,207,208,209,210,211,212,215,216,219,220,221,223,224,225,226,227,228,230,232,233,234,235,236,237,240,241,243,244,245,246,247,248,249,250,251,253,254,255,256,257,258,259,260,261,262,265,267,268,269,270,271,272,273,274,275,277,278,279,280,281,282,283,284,285,287,290,291,293,295,296,298,299,303,307,308,309,312) have mixed types. Specify dtype option on import or set low_memory=False.
  gene_presence_absence = pd.read_csv('pangenomes/panaroo_no_merge/gene_presence_absence_roar

,Gene,Isolate,locus_tag
0,lacF,UNY191,IBHJJELG_01239
1,guaB,UNY191,IBHJJELG_00988
2,group_1602,UNY191,IBHJJELG_01237
3,licB,UNY191,IBHJJELG_01238
4,guaA~~~guaA_2,UNY191,IBHJJELG_00989
...,...,...,...
379567,group_1101,UMA12,NDEJMIKF_00917
379568,group_81,UMA12,62_refound_2392
379569,group_700,UMA12,NDEJMIKF_01240
379570,group_90,UMA12,NDEJMIKF_01264


In [5]:
df = pd.merge(gene_presence_absence_melt, genes_by_contig, on=['Isolate', 'locus_tag'])
df

,Gene,Isolate,locus_tag,contig,genes
0,lacF,UNY191,IBHJJELG_01239,NODE_48,[IBHJJELG_01239]
1,guaB,UNY191,IBHJJELG_00988,contig_4_RagTag,[IBHJJELG_00988]
2,group_1602,UNY191,IBHJJELG_01237,NODE_48,[IBHJJELG_01237]
3,licB,UNY191,IBHJJELG_01238,NODE_48,[IBHJJELG_01238]
4,guaA~~~guaA_2,UNY191,IBHJJELG_00989,contig_4_RagTag,[IBHJJELG_00989]
...,...,...,...,...,...
361191,group_189,UMA12,NDEJMIKF_01013,NZ_CP161150.1_RagTag,[NDEJMIKF_01013]
361192,group_1101,UMA12,NDEJMIKF_00917,NZ_CP161146.1_RagTag,[NDEJMIKF_00917]
361193,group_700,UMA12,NDEJMIKF_01240,NODE_16,[NDEJMIKF_01240]
361194,group_90,UMA12,NDEJMIKF_01264,NODE_31,[NDEJMIKF_01264]


In [6]:
annotations = pd.read_csv('scaffolded_asms/replicon_annotations_1000bp.csv')
#annotations = pd.read_csv('../genotyping/replicons/calls_v10/best_hits_1000bp_v11.csv')
annotations['Isolate'] = annotations['assembly_id'].apply(lambda x: x.replace('_scaffold', ''))
annotations['contig'] = annotations['contig_id'].apply(lambda x: x.split('_length')[0])
annotations

,assembly_id,contig_id,contig_len,plasmid_id,plasmid_name,strain,query_length,ref_length,overall_percent_identity,query_covered_length,ref_covered_length,covered_intervals,query_intervals,subject_hit_coords,query_coverage_percent,call_method,Isolate,contig
0,ESI26_scaffold,NODE_11_length_22619_cov_115.895027,22619,RS07125_JD1_JD1_cp32-12_ParA_X,cp32-12,RS07125,22619,260,99.615385,779,259,"[(1, 260)]","[(3911, 4690)]","[(1, 260)]",3.444007,pf32,ESI26,NODE_11
1,ESI26_scaffold,NODE_14_length_8371_cov_131.134309,8371,gb|AE001580.1|,cp32-8,B31,8371,30885,96.116622,8258,8261,"[(10197, 15395), (7010, 10073)]","[(1, 3064), (3176, 8371)]","[(10197, 15395), (7010, 10073)]",98.650102,wp,ESI26,NODE_14
2,ESI26_scaffold,NODE_17_length_6020_cov_205.371390,6020,gb|CP001566.1|,lp28-9,Bol26,6020,28200,99.435216,6019,6019,"[(10083, 16102)]","[(1, 6020)]","[(10083, 16102)]",99.983389,wp,ESI26,NODE_17
3,ESI26_scaffold,NODE_21_length_4331_cov_126.964556,4331,RS01295_JD1_JD1_cp32-1+5_ParA_X,cp32-1+5,RS01295,4331,257,100.000000,770,256,"[(1, 257)]","[(1412, 2182)]","[(1, 257)]",17.778804,pf32,ESI26,NODE_21
4,ESI26_scaffold,NODE_23_length_4042_cov_213.095105,4042,gb|CP001566.1|,lp28-9,Bol26,4042,28200,98.892717,4041,4063,"[(20171, 24234)]","[(1, 4042)]","[(24234, 20171)]",99.975260,wp,ESI26,NODE_23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13159,UWI283_scaffold,contig_5_RagTag,32107,RS01730_MM1_plsm_cp32-9_ParA_X,cp32-9,RS01730,32107,251,100.000000,743,247,"[(1, 248)]","[(764, 1507)]","[(1, 248)]",2.314137,pf32,UWI283,contig_5_RagTag
13160,UWI283_scaffold,contig_6_RagTag,31682,RS05185_N40_N40_lp28-2_ParA_X,lp28-2,RS05185,31682,255,100.000000,764,254,"[(1, 255)]","[(24910, 25674)]","[(1, 255)]",2.411464,pf32,UWI283,contig_6_RagTag
13161,UWI283_scaffold,contig_7_RagTag,28921,I21_B31_lp28-4_ParA_X,lp28-4,I21,28921,250,100.000000,749,249,"[(1, 250)]","[(13619, 14368)]","[(1, 250)]",2.589814,pf32,UWI283,contig_7_RagTag
13162,UWI283_scaffold,contig_8_RagTag,11704,gb|CP001257.1|,lp54,156a,11704,53885,99.655321,11602,11602,"[(5218, 11675), (22, 5167)]","[(1, 6458), (6559, 11704)]","[(11675, 5218), (5167, 22)]",99.128503,wp,UWI283,contig_8_RagTag


In [7]:
contigs_genes_annotations = pd.merge(df[['Gene', 'Isolate', 'locus_tag', 'contig']], 
                                     annotations[['Isolate', 'contig_id', 'plasmid_id', 'plasmid_name', 'strain', 'call_method', 'contig']],
                                     on=['Isolate', 'contig'])
contigs_genes_annotations

,Gene,Isolate,locus_tag,contig,contig_id,plasmid_id,plasmid_name,strain,call_method
0,lacF,UNY191,IBHJJELG_01239,NODE_48,NODE_48_length_7717_cov_135.897,JD1_cp26_ParA_1,cp26,JD1,pf32
1,guaB,UNY191,IBHJJELG_00988,contig_4_RagTag,contig_4_RagTag,gb|CP001271.1|,cp26,156a,wp
2,group_1602,UNY191,IBHJJELG_01237,NODE_48,NODE_48_length_7717_cov_135.897,JD1_cp26_ParA_1,cp26,JD1,pf32
3,licB,UNY191,IBHJJELG_01238,NODE_48,NODE_48_length_7717_cov_135.897,JD1_cp26_ParA_1,cp26,JD1,pf32
4,guaA~~~guaA_2,UNY191,IBHJJELG_00989,contig_4_RagTag,contig_4_RagTag,gb|CP001271.1|,cp26,156a,wp
...,...,...,...,...,...,...,...,...,...
355952,group_189,UMA12,NDEJMIKF_01013,NZ_CP161150.1_RagTag,NZ_CP161150.1_RagTag,RS07610_JD1_JD1_lp28-6_ParA_X,lp28-6,RS07610,pf32
355953,group_1101,UMA12,NDEJMIKF_00917,NZ_CP161146.1_RagTag,NZ_CP161146.1_RagTag,RS00880_JD1_JD1_cp32-11_ParA_X,cp32-11,RS00880,pf32
355954,group_700,UMA12,NDEJMIKF_01240,NODE_16,NODE_16_length_12134_cov_1205.901482,gb|CP019764.1|,lp38,B31_NRZ,wp
355955,group_90,UMA12,NDEJMIKF_01264,NODE_31,NODE_31_length_4898_cov_1194.531695,gb|AE000787.1|,lp38,B31,wp


In [8]:
all_genes_a = sorted(list(contigs_genes_annotations['Gene'].unique()))
all_genes_b = sorted(list(df['Gene'].unique()))
all_genes_c = sorted(list(gene_presence_absence_melt['Gene'].unique()))
print(all_genes_a == all_genes_b)
print(all_genes_b == all_genes_c)
print(all_genes_a == all_genes_c)

False
True
False


In [9]:
all_loci_a = sorted(list(contigs_genes_annotations['locus_tag'].dropna().unique()))
all_loci_b = sorted(list(df['locus_tag'].dropna().unique()))
all_loci_c = sorted(list(genes_by_contig['locus_tag'].dropna().unique()))
print(all_loci_a == all_loci_b)
print(all_loci_b == all_loci_c)
print(all_loci_a == all_loci_c)

False
False
False


In [10]:
print(len(all_loci_a))
print(len(set(all_loci_a) - set(all_loci_b)))
print(len(set(all_loci_a) - set(all_loci_c)))
print()

print(len(all_loci_b))
print(len(set(all_loci_b) - set(all_loci_a)))
print(len(set(all_loci_b) - set(all_loci_c)))
print()

print(len(all_loci_c))
print(len(set(all_loci_c) - set(all_loci_a)))
print(len(set(all_loci_c) - set(all_loci_b)))

355957
0
0

361196
5239
0

379079
23122
17883


I think it's ok that there are missing locus_tags because many locus_tags are not of type=='CDS'?

In [11]:
contigs_genes_annotations.to_csv('contigs_genes_annotations_scaffolded_asms_panaroo_no_merge_20250213.csv', index=False)